In [ ]:
import quandl,  math, datetime
import pandas as pd
import numpy as np
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib import style
import pickle 



style.use('ggplot')






df = quandl.get('WIKI/GOOGL')

print(df.head())


In [ ]:
# we are creating the Features for our Regression algorithm

df = df[['Adj. Open',  'Adj. High',  'Adj. Low',  'Adj. Close', 'Adj. Volume',]]

df['HL_PCT'] = (df['Adj. High'] - df['Adj. Close']) / df['Adj. Close'] * 100.0

df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.0

df = df[['Adj. Close', 'HL_PCT', 'PCT_change', 'Adj. Volume',]]

print(df.head())



In [ ]:
forecast_col = 'Adj. Close'

# if there is missing data in our dataset

df.fillna('-99999', inplace=True)

forecast_out = int(math.ceil(0.1*len(df)))






In [ ]:
# creating the dataframe for the label

df['label'] = df[forecast_col].shift(-forecast_out)




# print(df.tail())

print(df.head())




In [ ]:
# lets scale our Features

X = preprocessing.scale(X)
X_lately = X[-forecast_out:]
X = X[:-forecast_out:]

# lets print our new output

df.dropna(inplace=True)

y = np.array(df['label'])
y = np.array(df['label'])


In [ ]:
# features are represented by Letter X and labels by y

X = np.array(df.drop(['label'], 1))




In [ ]:

# print(len(X),len(y))

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2)




In [ ]:
# for the classifiers for our features and labels

clf = LinearRegression()

# clf = svm.SVR(kernel='poly')

# for training
clf.fit(X_train, y_train)

# lets use pickle to save our algorithm model

# with open('linearregression.pickle','wb') as f:
#      pickle.dumps(clf, f)

# pickle_in = open('linearregression.pickle', 'wb')
# clf = pickle.load(pickle_in)


# for testing
accuracy = clf.score(X_test, y_test)

# lets print to show the accuracy of our model
# print(accuracy)




In [ ]:
# lets start our prediction

forecast_set = clf.predict(X_lately)

# lets print the forecast_set

print(forecast_set, accuracy, forecast_out)



In [ ]:
# lets do some stuff

df['Forecast'] = np.nan

last_date = df.iloc[-1].name

last_unix = last_date.timestamp()

one_day = 86400

next_unix = last_unix + one_day

for i in forecast_set:
    next_date = datetime.datetime.fromtimestamp(next_unix)
    next_unix += one_day
    df.loc[next_date] = [np.nan for _ in range(len(df.columns)-1)] + [i]
    
# lets do some plotting
    
df['Adj. Close'].plot()
df['Forecast'].plot()
plt.legend(loc=4)
plt.xlabel('Date')
plt.ylabel('Price')



